---

# 🧮 Estimación de Puntuaciones de Examen con Regresión Lineal

## 📘 Informe de Investigación 3
**Materia:** Inteligencia de Negocios
**Tema:** Estimación usando Regresión Lineal

**Autores:**
- Randy Alexander Germosén Ureña *(1013-4707)*
- Fernando Almonte Delgado *(1015-7628)*

---

## 🎯 Objetivo del Proyecto
El propósito de este trabajo es aplicar la **Regresión Lineal** como técnica de estimación para predecir la **puntuación en un examen** de un estudiante en función de dos variables predictoras:
- Horas de estudio
- Horas de sueño

A través de este modelo, se busca analizar cómo estos factores influyen en el desempeño académico y evaluar la precisión del modelo predictivo mediante métricas de desempeño.

---

## 📂 Dataset Utilizado
**Archivo:** `student_exam_regression.csv`

---

## 🧠 Metodología


---

## 📊 Resultados


---

## 🧩 Herramientas Utilizadas
- **R** (en un entorno de Notebook de Jupyter o RStudio)
- **Librerías principales (R):**
  - `readr` y `dplyr` (parte de `tidyverse`) – Manejo y análisis de datos
  - `ggplot2` (parte de `tidyverse`) – Visualización de datos
  - `GGally` – Matriz de gráficos de correlación (EDA)
  - `caTools` – División de datos (train/test split)